In [1]:
# Imports for Gemini (LangChain) in the same structure as OpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain  # keep if you still use it elsewhere
from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks import StdOutCallbackHandler
# Note: LLMChain is deprecated; use Prompt | llm | StrOutputParser instead.

import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

# Load environment variables from .env (expects GOOGLE_API_KEY)
load_dotenv()


True

In [3]:
KEY = os.getenv("GOOGLE_API_KEY")

In [8]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.6, google_api_key=KEY)
print(llm.invoke("Reply with OK").content)

OK


In [5]:
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.6, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000283B80A3380>, default_metadata=())

In [9]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [6]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [7]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [39]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [40]:
TEMPLATE="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [41]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [42]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [43]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [44]:

"""pdf_reader=PyPDF2.PdfReader(r"C:\Complete_Content\All_Project\TEST_FOR_EVERYTHING\langchain\experiments\Big Data Assessment.pdf")
text = ""
for page in pdf_reader.pages:
    text += page.extract_text()"""

<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Rajat Pandey\AppData\Local\Temp\ipykernel_34732\2136195752.py:1: SyntaxWarning: invalid escape sequence '\C'
  """pdf_reader=PyPDF2.PdfReader(r"C:\Complete_Content\All_Project\TEST_FOR_EVERYTHING\langchain\experiments\Big Data Assessment.pdf")


'pdf_reader=PyPDF2.PdfReader(r"C:\\Complete_Content\\All_Project\\TEST_FOR_EVERYTHING\\langchain\\experiments\\Big Data Assessment.pdf")\ntext = ""\nfor page in pdf_reader.pages:\n    text += page.extract_text()'

In [46]:
file_path = r"C:\Users\Rajat Pandey\Desktop\AI Engineering\mcqgen_0.1\data.txt"

In [47]:
file_path

'C:\\Users\\Rajat Pandey\\Desktop\\AI Engineering\\mcqgen_0.1\\data.txt'

In [48]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [50]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[6][7] The synonym self-teaching computers was also used in this time period.[8][9]

The earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[10] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[11] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[10] Other researchers who have studied human cognitive systems contribu

In [51]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [52]:
"""mcq_count=5
grade=1
tone="simple"""

'mcq_count=5\ngrade=1\ntone="simple'

In [53]:
NUMBER=5 
SUBJECT="data science"
TONE="simple"

In [57]:
from langchain.callbacks import StdOutCallbackHandler

handler = StdOutCallbackHandler()
payload = {
  "text": TEXT, "number": NUMBER, "subject": SUBJECT,
  "tone": TONE, "response_json": json.dumps(RESPONSE_JSON)
}
# for LCEL/chains:
out = generate_evaluate_chain.invoke(payload, config={"callbacks": [handler]})
print(out)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[6][7] The synonym self-teaching computers was also used in this time period.[8][9]

The earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[10] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[11] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons us

In [63]:
from langchain.callbacks.base import BaseCallbackHandler

class SaveLLMResponseHandler(BaseCallbackHandler):
    def __init__(self):
        self.last_response = None
    def on_llm_end(self, response, **kwargs):
        self.last_response = response


def extract_usage_from_response(resp):
    if resp is None:
        return None
    meta = getattr(resp, "response_metadata", None) or (resp.get("response_metadata") if isinstance(resp, dict) else None)
    if not meta:
        return None
    return meta.get("usage_metadata") or meta.get("usage") or meta.get("token_count") or None


def print_usage(usage):
    if not usage:
        print("No usage metadata found.")
        return
    total = usage.get("total_tokens") or usage.get("total_token_count") or usage.get("total") or usage.get("tokens")
    prompt = usage.get("prompt_tokens") or usage.get("input_token_count") or usage.get("input_tokens")
    completion = usage.get("completion_tokens") or usage.get("output_token_count") or usage.get("output_tokens")
    cost = usage.get("total_cost") or usage.get("cost")
    print("usage raw:", usage)
    print(f"Total tokens: {total}")
    print(f"Prompt tokens: {prompt}")
    print(f"Completion tokens: {completion}")
    print(f"Cost (if provided): {cost}")


In [64]:
# Run the chain and capture provider response + usage
save_handler = SaveLLMResponseHandler()
payload = {"text": TEXT, "number": NUMBER, "subject": SUBJECT, "tone": TONE, "response_json": json.dumps(RESPONSE_JSON)}
out = generate_evaluate_chain.invoke(payload, config={"callbacks": [handler, save_handler]})
print(out)
resp = getattr(save_handler, 'last_response', None)
usage = extract_usage_from_response(resp)
print_usage(usage)




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[6][7] The synonym self-teaching computers was also used in this time period.[8][9]

The earliest machine learning program was introduced in the 1950s when Arthur Samuel invented a computer program that calculated the winning chance in checkers for each side, but the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[10] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[11] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons us

In [72]:
# A: locate the quiz string in common variables
quiz_str = None
if 'out' in globals() and isinstance(out, dict):
    quiz_str = out.get('quiz')
elif 'resp' in globals():
    if isinstance(resp, dict):
        quiz_str = resp.get('quiz') or resp.get('content')
    else:
        quiz_str = getattr(resp, 'content', None)

if quiz_str is None:
    raise NameError("No quiz found in 'out', 'response', or 'resp'. Run the chain and retry.")

# B: parse quiz_str into quiz_dict (robust JSON extraction)
import json, re
quiz_dict = None
if isinstance(quiz_str, dict):
    quiz_dict = quiz_str
else:
    try:
        quiz_dict = json.loads(quiz_str)
    except Exception:
        m = re.search(r'(\{.*\})', str(quiz_str), re.DOTALL)
        if m:
            try:
                quiz_dict = json.loads(m.group(1))
            except Exception as e:
                raise ValueError('Failed to parse JSON from quiz_str: ' + str(e))
        else:
            raise ValueError('quiz_str is not valid JSON and no JSON substring found.')
quiz_dict = {str(k): v for k, v in quiz_dict.items()}  # normalize keys to strings

# C: display parsed quiz_dict
quiz_dict


{'1': {'mcq': "Who coined the term 'machine learning'?",
  'options': {'a': 'Donald Hebb',
   'b': 'Alan Turing',
   'c': 'Arthur Samuel',
   'd': 'Tom M. Mitchell'},
  'correct': 'c'},
 '2': {'mcq': 'What was an early application of machine learning mentioned in the text?',
  'options': {'a': 'Predicting the weather',
   'b': 'Analyzing sonar signals',
   'c': 'Translating languages',
   'd': 'Playing chess'},
  'correct': 'b'},
 '3': {'mcq': 'Which book introduced a theoretical neural structure that influenced machine learning?',
  'options': {'a': 'Learning Machines by Nilsson',
   'b': 'Computing Machinery and Intelligence by Turing',
   'c': 'The Organization of Behavior by Hebb',
   'd': 'A book on pattern recognition by Duda and Hart'},
  'correct': 'c'},
 '4': {'mcq': "According to Tom M. Mitchell's definition, what improves with experience in machine learning?",
  'options': {'a': 'The complexity of the algorithms',
   'b': 'The size of the dataset',
   'c': 'Performance at ta

In [73]:
quiz_table_data = []
for key, value in quiz_dict.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [74]:
quiz_table_data

[{'MCQ': "Who coined the term 'machine learning'?",
  'Choices': 'a: Donald Hebb | b: Alan Turing | c: Arthur Samuel | d: Tom M. Mitchell',
  'Correct': 'c'},
 {'MCQ': 'What was an early application of machine learning mentioned in the text?',
  'Choices': 'a: Predicting the weather | b: Analyzing sonar signals | c: Translating languages | d: Playing chess',
  'Correct': 'b'},
 {'MCQ': 'Which book introduced a theoretical neural structure that influenced machine learning?',
  'Choices': 'a: Learning Machines by Nilsson | b: Computing Machinery and Intelligence by Turing | c: The Organization of Behavior by Hebb | d: A book on pattern recognition by Duda and Hart',
  'Correct': 'c'},
 {'MCQ': "According to Tom M. Mitchell's definition, what improves with experience in machine learning?",
  'Choices': 'a: The complexity of the algorithms | b: The size of the dataset | c: Performance at tasks | d: The number of parameters',
  'Correct': 'c'},
 {'MCQ': 'Which of the following is NOT a mode

In [75]:
df=pd.DataFrame(quiz_table_data)

In [76]:
df

,MCQ,Choices,Correct
0,Who coined the term 'machine learning'?,a: Donald Hebb | b: Alan Turing | c: Arthur Sa...,c
1,What was an early application of machine learn...,a: Predicting the weather | b: Analyzing sonar...,b
2,Which book introduced a theoretical neural str...,a: Learning Machines by Nilsson | b: Computing...,c
3,"According to Tom M. Mitchell's definition, wha...",a: The complexity of the algorithms | b: The s...,c
4,Which of the following is NOT a modern objecti...,a: Classifying data | b: Making predictions | ...,c


In [77]:
df.to_csv("Data_Science_Quiz.csv",index=False)

In [78]:
import traceback
try:
    c=1/0
except Exception as e:
        traceback.print_exception(type(e), e, e.__traceback__)

Traceback (most recent call last):
  File "C:\Users\Rajat Pandey\AppData\Local\Temp\ipykernel_34732\2720247763.py", line 3, in <module>
    c=1/0
      ~^~
ZeroDivisionError: division by zero


In [79]:
import traceback
try:
    c=1/0
except Exception as e:
    tb=e.__traceback__

In [80]:
while tb is not None:
    frame = tb.tb_frame
    filename = frame.f_code.co_filename
    lineno = frame.f_lineno
    function_name = frame.f_code.co_name

    print(f"File '{filename}', line {lineno}, in {function_name}")
    tb = tb.tb_next  # Move to the next frame in the traceback

File 'C:\Users\Rajat Pandey\AppData\Local\Temp\ipykernel_34732\977600961.py', line 5, in <module>
